## K-means
**time : 2019/12/25**   
**author: mjl**  

## 1.算法原理
k-means是最普及的聚类算法，算法接受一个来标记的数据集，然后将数据聚类成不同的组。  

使用$\mu^{1},\mu^{2},\mu^{3},...,\mu^{k}$来表示聚类中心，用$c^{1},c^{2},c^{3},...,c^{m}$来存储与第i个实例数据最近的聚类中心的索引。  

k-means的优化目标，其实是最小化所有数据点与其所关联的聚类中心点之间的距离之和。即k-means的代价函数：  

$J(c^{1},c^{2},c^{3},...,c^{m}, \mu^{1},\mu^{2},\mu^{3},...,\mu^{k}) = \frac{1}{m}\sum \left \| x^{i} - \mu^{c_{i}} \right \|^{2}$  

其中$\mu^{i}$代表与$x_{i}$最近的聚类中心点，我们的优化目标便是找出使得代价函数最小的$c^{1},c^{2},c^{3},...,c^{m}$和$\mu^{1},\mu^{2},\mu^{3},...,\mu^{k}$。

## 2.算法实现
k-means是最普及的聚类算法，算法接受一个来标记的数据集，然后将数据聚类成不同的组。算法分为两个步骤，第一个for循环是赋值步骤，即对于每一个样例i，计算其应该属于的类。第二个for循环是聚类中心的移动，即对于每一个类k，重新计算该类的中心。  
Repeat {  
&emsp;&emsp;  for i = 1 : m      
&emsp;&emsp;&emsp;&emsp; $C^{i}$ = index (from 1 to k) of cluster centroid  
      
&emsp;&emsp;  for k = 1 : K  
&emsp;&emsp;&emsp;&emsp; $\mu^{k}$ = average (mean) of points assigned to cluster k  
}


## 3.导包

In [14]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
config = tf.ConfigProto()

config.gpu_options.allow_growth = True

session = tf.Session(config=config)

## 4.代码实现

In [25]:
def accuary(targets, labels):
    return np.mean(targets == labels, axis=0)

def distance(x1, x2):
    return np.power(np.sum(np.power(x1 - x2, 2)), 0.5)

def kmeans(data, labels, classs = 12, epoch=100):
    ## 记录每个数据的中心
    pos = [i for i in range(0, len(labels))]
    ## 聚类中心， 初始时进行随机初始化
    center = []
    for i in random.sample(range(0, len(labels)), classs):
        center.append(data[i, :])
    center = np.array(center)
    
    for m in range(epoch):
        for i, x1 in enumerate(data):
            diss = []
            ## 计算每个点到所有中心的距离
            for x2 in center:
                diss.append(distance(x1, x2))
            ## 将数据聚类到距离其最小的中心
            pos[i] = np.argmin(diss)
        ## 下面是通过前面聚类到的数据， 对中心进行更新
        for i in range(classs):
            center[i] = 0
        for i, x in enumerate(data):
            center[pos[i]] += x
        ## 这里， 中心的更新， 就是对属于其的数据的坐标进行平均
        for i in range(classs):
            center[i] = center[i] / np.sum(np.array(pos) == i)
    return center

def predict(x, y, classs, center):
    pos = [i for i in range(0, x.shape[0])]
    for i, x1 in enumerate(x):
        diss = []
        for x2 in center:
            diss.append(distance(x1, x2))
        pos[i] = np.argmin(diss)
    print(accuary(np.array(pos).reshape([-1, 1]), y))

## 5.导入数据

In [26]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

scaler = StandardScaler()
X1 = scaler.fit_transform(load_breast_cancer()['data'])
Y1 = load_breast_cancer()['target'].reshape([-1, 1])
print(X1.shape)
print(Y1.shape)

(569, 30)
(569, 1)


## 6.测试数据

In [27]:
xtrain, xtest, ytrain, ytest = train_test_split(X1, Y1, test_size=.2)
center = kmeans(X1, Y1, 12, 200)
predict(X1, Y1, 12, center)

print("官方提供：")
kmeans = KMeans(n_clusters=12)
kmeans.fit(X1)
pre = kmeans.predict(X1)
print(accuary(pre.reshape([-1,1]), Y1))

[ 0.09841828]
官方提供：
[ 0.02636204]
